In [1]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
DATA_DIR = "C:/Users/91701/Major demo1/H"
OUTPUT_DIR = "C:/Users/91701/Major demo1/outputs"
CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, "checkpoints")
LOG_DIR = os.path.join(OUTPUT_DIR, "logs")

In [3]:
IMAGE_SIZE = 128
BATCH_SIZE = 8
EPOCHS = 700
LATENT_DIM = 100
N_CRITIC = 5  # Discriminator updates per generator update
LAMBDA_GP = 10  # Gradient penalty weight
LEARNING_RATE = 5e-5
G_LEARNING_RATE = 1e-4

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [4]:
device_name = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
print(f"Using device: {device_name}")

Using device: CPU


In [5]:
def load_images():
    images = []
    for file in os.listdir(DATA_DIR):
        if file.lower().endswith(('png','jpg','jpeg')):
            img = Image.open(os.path.join(DATA_DIR, file)).convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))
            img = np.array(img, dtype=np.float32)
            img = (img / 127.5) - 1.0  # Normalize to [-1, 1]
            images.append(img)
    return np.array(images)

images_np = load_images()
print(f"Loaded dataset: {images_np.shape}")

dataset = tf.data.Dataset.from_tensor_slices(images_np)

Loaded dataset: (227, 128, 128, 3)


In [6]:
def augment(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image

dataset = dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=len(images_np)).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [7]:
def build_generator(latent_dim):
    model = tf.keras.Sequential([
        layers.Input(shape=(latent_dim,)),
        layers.Dense(8*8*256, use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),

        layers.Conv2DTranspose(128, 5, strides=2, padding="same", use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(),

        layers.Conv2DTranspose(64, 5, strides=2, padding="same", use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(),

        layers.Conv2DTranspose(32, 5, strides=2, padding="same", use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(),

        layers.Conv2DTranspose(3, 5, strides=2, padding="same", activation='tanh')
    ])
    return model

In [8]:
def build_discriminator(img_shape):
    model = tf.keras.Sequential([
        layers.Input(shape=img_shape),
        layers.Conv2D(64, 5, strides=2, padding="same"),
        layers.LeakyReLU(0.2),

        layers.Conv2D(128, 5, strides=2, padding="same"),
        layers.LayerNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(256, 5, strides=2, padding="same"),
        layers.LayerNormalization(),
        layers.LeakyReLU(0.2),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

generator = build_generator(LATENT_DIM)
discriminator = build_discriminator((IMAGE_SIZE, IMAGE_SIZE, 3))


In [9]:
def gradient_penalty(batch_size, real_images, fake_images):
    alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
    interpolated = alpha * real_images + (1 - alpha) * fake_images

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        pred = discriminator(interpolated, training=True)
    grads = tape.gradient(pred, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

def generator_loss(fake_pred):
    return -tf.reduce_mean(fake_pred)

def discriminator_loss(real_pred, fake_pred):
    return tf.reduce_mean(fake_pred) - tf.reduce_mean(real_pred)

In [10]:
generator_optimizer = tf.keras.optimizers.Adam(G_LEARNING_RATE, beta_1=0.0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.0, beta_2=0.9)

In [11]:
checkpoint = tf.train.Checkpoint(generator=generator, discriminator=discriminator,
                                 g_optimizer=generator_optimizer, d_optimizer=discriminator_optimizer)
manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=5)

summary_writer = tf.summary.create_file_writer(LOG_DIR)

In [12]:
@tf.function
def train_step(real_images):
    batch_size = tf.shape(real_images)[0]

    for _ in range(N_CRITIC):
        noise = tf.random.normal([batch_size, LATENT_DIM])

        with tf.GradientTape() as disc_tape:
            fake_images = generator(noise, training=True)
            real_pred = discriminator(real_images, training=True)
            fake_pred = discriminator(fake_images, training=True)

            gp = gradient_penalty(batch_size, real_images, fake_images)
            d_loss = discriminator_loss(real_pred, fake_pred) + LAMBDA_GP * gp

        d_grad = disc_tape.gradient(d_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(d_grad, discriminator.trainable_variables))

    noise = tf.random.normal([batch_size, LATENT_DIM])
    with tf.GradientTape() as gen_tape:
        fake_images = generator(noise, training=True)
        fake_pred = discriminator(fake_images, training=True)
        g_loss = generator_loss(fake_pred)

    g_grad = gen_tape.gradient(g_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(g_grad, generator.trainable_variables))

    return g_loss, d_loss

In [13]:
def generate_and_save_images(epoch):
    test_input = tf.random.normal([16, LATENT_DIM])
    predictions = generator(test_input, training=False)
    predictions = (predictions + 1.0) / 2.0

    grid_img = np.zeros((IMAGE_SIZE*4, IMAGE_SIZE*4, 3))
    for i in range(4):
        for j in range(4):
            img = predictions[i*4+j].numpy()
            grid_img[i*IMAGE_SIZE:(i+1)*IMAGE_SIZE, j*IMAGE_SIZE:(j+1)*IMAGE_SIZE, :] = img

    Image.fromarray((grid_img*255).astype(np.uint8)).save(os.path.join(OUTPUT_DIR, f"epoch_{epoch:03d}.png"))

In [15]:
for epoch in range(1, EPOCHS+1):
    g_losses, d_losses = [], []
    for real_images in tqdm(dataset, desc=f"Epoch {epoch}/{EPOCHS}"):
        g_loss, d_loss = train_step(real_images)
        g_losses.append(g_loss.numpy())
        d_losses.append(d_loss.numpy())

    with summary_writer.as_default():
        tf.summary.scalar('Generator Loss', np.mean(g_losses), step=epoch)
        tf.summary.scalar('Discriminator Loss', np.mean(d_losses), step=epoch)

    if epoch % 5 == 0:
        generate_and_save_images(epoch)
        manager.save()

    print(f"Epoch {epoch} | Gen Loss: {np.mean(g_losses):.4f} | Disc Loss: {np.mean(d_losses):.4f}")

Epoch 1/500: 100%|█████████████████████████████████████████████████████████████████████| 28/28 [02:04<00:00,  4.46s/it]


Epoch 1 | Gen Loss: 113.8449 | Disc Loss: -179.4870


Epoch 2/500: 100%|█████████████████████████████████████████████████████████████████████| 28/28 [01:56<00:00,  4.15s/it]


Epoch 2 | Gen Loss: 212.4664 | Disc Loss: -263.2340


Epoch 3/500: 100%|█████████████████████████████████████████████████████████████████████| 28/28 [01:43<00:00,  3.68s/it]


...

Epoch 119 | Gen Loss: 309.3061 | Disc Loss: -28.3511


Epoch 120/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:40<00:00,  3.59s/it]


Epoch 120 | Gen Loss: 309.5085 | Disc Loss: -30.3601


Epoch 121/500:   4%|██▍                                                                 | 1/28 [00:08<03:38,  8.07s/it]


KeyboardInterrupt: 

In [22]:
# === Create models and optimizers exactly as before ===
generator = build_generator(LATENT_DIM)
discriminator = build_discriminator((IMAGE_SIZE, IMAGE_SIZE, 3))
generator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.0, beta_2=0.9)

# === Set up checkpoint ===
checkpoint = tf.train.Checkpoint(
    generator=generator,
    discriminator=discriminator,
    g_optimizer=generator_optimizer,
    d_optimizer=discriminator_optimizer
)
manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=5)

# === Restore ===
if manager.latest_checkpoint:
    checkpoint.restore(manager.latest_checkpoint).expect_partial()
    print(f"Restored from {manager.latest_checkpoint}")

    # Extract the save number from ckpt-XX and convert to epoch
    save_num = int(manager.latest_checkpoint.split('-')[-1])
    start_epoch = save_num * 5 + 1  # resume from next epoch
    print(f"Resuming from epoch {start_epoch}")
else:
    print("No checkpoint found, starting from scratch.")
    start_epoch = 1

# === Training Loop ===
for epoch in range(start_epoch, EPOCHS+1):
    g_losses, d_losses = [], []
    for real_images in tqdm(dataset, desc=f"Epoch {epoch}/{EPOCHS}"):
        g_loss, d_loss = train_step(real_images)
        g_losses.append(g_loss.numpy())
        d_losses.append(d_loss.numpy())

    with summary_writer.as_default():
        tf.summary.scalar('Generator Loss', np.mean(g_losses), step=epoch)
        tf.summary.scalar('Discriminator Loss', np.mean(d_losses), step=epoch)

    if epoch % 5 == 0:
        generate_and_save_images(epoch)
        manager.save()

    print(f"Epoch {epoch} | Gen Loss: {np.mean(g_losses):.4f} | Disc Loss: {np.mean(d_losses):.4f}")


Restored from C:/Users/91701/Major demo1/outputs/checkpoints\ckpt-24
Resuming from epoch 121


Epoch 121/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:41<00:00,  3.64s/it]


Epoch 121 | Gen Loss: 308.6078 | Disc Loss: -30.8166


Epoch 122/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:38<00:00,  3.51s/it]


Epoch 122 | Gen Loss: 311.8988 | Disc Loss: -32.4981


...

Epoch 224 | Gen Loss: 316.7945 | Disc Loss: -33.7021


Epoch 225/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:43<00:00,  3.68s/it]


Epoch 225 | Gen Loss: 314.9870 | Disc Loss: -34.7852


Epoch 226/500:  21%|██████████████▌                                                     | 6/28 [00:28<01:46,  4.82s/it]


KeyboardInterrupt: 

In [37]:
# === Create models and optimizers exactly as before ===
generator = build_generator(LATENT_DIM)
discriminator = build_discriminator((IMAGE_SIZE, IMAGE_SIZE, 3))
generator_optimizer = tf.keras.optimizers.Adam(G_LEARNING_RATE, beta_1=0.0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.0, beta_2=0.9)

# === Set up checkpoint ===
checkpoint = tf.train.Checkpoint(
    generator=generator,
    discriminator=discriminator,
    g_optimizer=generator_optimizer,
    d_optimizer=discriminator_optimizer
)
manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=5)

# === Restore ===
if manager.latest_checkpoint:
    checkpoint.restore(manager.latest_checkpoint).expect_partial()
    print(f"Restored from {manager.latest_checkpoint}")

    # Extract the save number from ckpt-XX and convert to epoch
    save_num = int(manager.latest_checkpoint.split('-')[-1])
    start_epoch = save_num * 5 + 1  # resume from next epoch
    print(f"Resuming from epoch {start_epoch}")
else:
    print("No checkpoint found, starting from scratch.")
    start_epoch = 1

# === Training Loop ===
for epoch in range(start_epoch, EPOCHS+1):
    g_losses, d_losses = [], []
    for real_images in tqdm(dataset, desc=f"Epoch {epoch}/{EPOCHS}"):
        g_loss, d_loss = train_step(real_images)
        g_losses.append(g_loss.numpy())
        d_losses.append(d_loss.numpy())

    with summary_writer.as_default():
        tf.summary.scalar('Generator Loss', np.mean(g_losses), step=epoch)
        tf.summary.scalar('Discriminator Loss', np.mean(d_losses), step=epoch)

    if epoch % 5 == 0:
        generate_and_save_images(epoch)
        manager.save()

    print(f"Epoch {epoch} | Gen Loss: {np.mean(g_losses):.4f} | Disc Loss: {np.mean(d_losses):.4f}")


Restored from C:/Users/91701/Major demo1/outputs\checkpoints\ckpt-45
Resuming from epoch 226


Epoch 226/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:51<00:00,  3.99s/it]


Epoch 226 | Gen Loss: 307.7602 | Disc Loss: -32.3321


Epoch 227/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:37<00:00,  3.49s/it]


Epoch 227 | Gen Loss: 312.5345 | Disc Loss: -30.8890


...

Epoch 239 | Gen Loss: 297.6432 | Disc Loss: -32.2526


Epoch 240/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:37<00:00,  3.47s/it]


Epoch 240 | Gen Loss: 299.4858 | Disc Loss: -29.1519


Epoch 241/500:  64%|███████████████████████████████████████████                        | 18/28 [01:11<00:39,  3.99s/it]


KeyboardInterrupt: 

In [14]:
# === Create models and optimizers exactly as before ===
generator = build_generator(LATENT_DIM)
discriminator = build_discriminator((IMAGE_SIZE, IMAGE_SIZE, 3))
generator_optimizer = tf.keras.optimizers.Adam(G_LEARNING_RATE, beta_1=0.0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.0, beta_2=0.9)

# === Set up checkpoint ===
checkpoint = tf.train.Checkpoint(
    generator=generator,
    discriminator=discriminator,
    g_optimizer=generator_optimizer,
    d_optimizer=discriminator_optimizer
)
manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=5)

# === Restore ===
if manager.latest_checkpoint:
    checkpoint.restore(manager.latest_checkpoint).expect_partial()
    print(f"Restored from {manager.latest_checkpoint}")

    # Extract the save number from ckpt-XX and convert to epoch
    save_num = int(manager.latest_checkpoint.split('-')[-1])
    start_epoch = save_num * 5 + 1  # resume from next epoch
    print(f"Resuming from epoch {start_epoch}")
else:
    print("No checkpoint found, starting from scratch.")
    start_epoch = 1

# === Training Loop ===
for epoch in range(start_epoch, EPOCHS+1):
    g_losses, d_losses = [], []
    for real_images in tqdm(dataset, desc=f"Epoch {epoch}/{EPOCHS}"):
        g_loss, d_loss = train_step(real_images)
        g_losses.append(g_loss.numpy())
        d_losses.append(d_loss.numpy())

    with summary_writer.as_default():
        tf.summary.scalar('Generator Loss', np.mean(g_losses), step=epoch)
        tf.summary.scalar('Discriminator Loss', np.mean(d_losses), step=epoch)

    if epoch % 5 == 0:
        generate_and_save_images(epoch)
        manager.save()

    print(f"Epoch {epoch} | Gen Loss: {np.mean(g_losses):.4f} | Disc Loss: {np.mean(d_losses):.4f}")


Restored from C:/Users/91701/Major demo1/outputs\checkpoints\ckpt-48
Resuming from epoch 241


Epoch 241/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:55<00:00,  4.11s/it]


Epoch 241 | Gen Loss: 295.8997 | Disc Loss: -34.9835


Epoch 242/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:40<00:00,  3.59s/it]


Epoch 242 | Gen Loss: 295.4944 | Disc Loss: -32.2458


...

Epoch 294 | Gen Loss: 307.2088 | Disc Loss: -30.1168


Epoch 295/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:46<00:00,  3.82s/it]


Epoch 295 | Gen Loss: 300.7493 | Disc Loss: -32.1466


Epoch 296/500:  36%|███████████████████████▉                                           | 10/28 [00:43<01:18,  4.37s/it]


KeyboardInterrupt: 

In [14]:
# === Create models and optimizers exactly as before ===
generator = build_generator(LATENT_DIM)
discriminator = build_discriminator((IMAGE_SIZE, IMAGE_SIZE, 3))
generator_optimizer = tf.keras.optimizers.Adam(G_LEARNING_RATE, beta_1=0.0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.0, beta_2=0.9)

# === Set up checkpoint ===
checkpoint = tf.train.Checkpoint(
    generator=generator,
    discriminator=discriminator,
    g_optimizer=generator_optimizer,
    d_optimizer=discriminator_optimizer
)
manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=5)

# === Restore ===
if manager.latest_checkpoint:
    checkpoint.restore(manager.latest_checkpoint).expect_partial()
    print(f"Restored from {manager.latest_checkpoint}")

    # Extract the save number from ckpt-XX and convert to epoch
    save_num = int(manager.latest_checkpoint.split('-')[-1])
    start_epoch = save_num * 5 + 1  # resume from next epoch
    print(f"Resuming from epoch {start_epoch}")
else:
    print("No checkpoint found, starting from scratch.")
    start_epoch = 1

# === Training Loop ===
for epoch in range(start_epoch, EPOCHS+1):
    g_losses, d_losses = [], []
    for real_images in tqdm(dataset, desc=f"Epoch {epoch}/{EPOCHS}"):
        g_loss, d_loss = train_step(real_images)
        g_losses.append(g_loss.numpy())
        d_losses.append(d_loss.numpy())

    with summary_writer.as_default():
        tf.summary.scalar('Generator Loss', np.mean(g_losses), step=epoch)
        tf.summary.scalar('Discriminator Loss', np.mean(d_losses), step=epoch)

    if epoch % 5 == 0:
        generate_and_save_images(epoch)
        manager.save()

    print(f"Epoch {epoch} | Gen Loss: {np.mean(g_losses):.4f} | Disc Loss: {np.mean(d_losses):.4f}")


Restored from C:/Users/91701/Major demo1/outputs\checkpoints\ckpt-59
Resuming from epoch 296


Epoch 296/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [02:02<00:00,  4.39s/it]


Epoch 296 | Gen Loss: 311.9011 | Disc Loss: -33.6586


Epoch 297/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:45<00:00,  3.78s/it]


Epoch 297 | Gen Loss: 308.8129 | Disc Loss: -33.4346


...

Epoch 498 | Gen Loss: 400.1504 | Disc Loss: -33.6495


Epoch 499/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:36<00:00,  3.44s/it]


Epoch 499 | Gen Loss: 407.6773 | Disc Loss: -34.5692


Epoch 500/500: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:37<00:00,  3.47s/it]


Epoch 500 | Gen Loss: 399.6084 | Disc Loss: -32.2891


In [14]:
# === Create models and optimizers exactly as before ===
generator = build_generator(LATENT_DIM)
discriminator = build_discriminator((IMAGE_SIZE, IMAGE_SIZE, 3))
generator_optimizer = tf.keras.optimizers.Adam(G_LEARNING_RATE, beta_1=0.0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.0, beta_2=0.9)

# === Set up checkpoint ===
checkpoint = tf.train.Checkpoint(
    generator=generator,
    discriminator=discriminator,
    g_optimizer=generator_optimizer,
    d_optimizer=discriminator_optimizer
)
manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=5)

# === Restore ===
if manager.latest_checkpoint:
    checkpoint.restore(manager.latest_checkpoint).expect_partial()
    print(f"Restored from {manager.latest_checkpoint}")

    # Extract the save number from ckpt-XX and convert to epoch
    save_num = int(manager.latest_checkpoint.split('-')[-1])
    start_epoch = save_num * 5 + 1  # resume from next epoch
    print(f"Resuming from epoch {start_epoch}")
else:
    print("No checkpoint found, starting from scratch.")
    start_epoch = 1

# === Training Loop ===
for epoch in range(start_epoch, EPOCHS+1):
    g_losses, d_losses = [], []
    for real_images in tqdm(dataset, desc=f"Epoch {epoch}/{EPOCHS}"):
        g_loss, d_loss = train_step(real_images)
        g_losses.append(g_loss.numpy())
        d_losses.append(d_loss.numpy())

    with summary_writer.as_default():
        tf.summary.scalar('Generator Loss', np.mean(g_losses), step=epoch)
        tf.summary.scalar('Discriminator Loss', np.mean(d_losses), step=epoch)

    if epoch % 5 == 0:
        generate_and_save_images(epoch)
        manager.save()

    print(f"Epoch {epoch} | Gen Loss: {np.mean(g_losses):.4f} | Disc Loss: {np.mean(d_losses):.4f}")


Restored from C:/Users/91701/Major demo1/outputs\checkpoints\ckpt-100
Resuming from epoch 501


Epoch 501/700: 100%|███████████████████████████████████████████████████████████████████| 28/28 [02:00<00:00,  4.31s/it]


Epoch 501 | Gen Loss: 401.1227 | Disc Loss: -34.1951


Epoch 502/700: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:37<00:00,  3.50s/it]


Epoch 502 | Gen Loss: 401.5496 | Disc Loss: -33.4092


...

Epoch 656 | Gen Loss: 485.1056 | Disc Loss: -32.2131


Epoch 657/700: 100%|███████████████████████████████████████████████████████████████████| 28/28 [01:38<00:00,  3.52s/it]


Epoch 657 | Gen Loss: 493.8218 | Disc Loss: -35.0385


Epoch 658/700:  96%|████████████████████████████████████████████████████████████████▌  | 27/28 [01:43<00:03,  3.85s/it]


KeyboardInterrupt: 